# Python для анализа данных

# Spark / PySpark 

#### автор: Валентин Бирюков


Spark является все более популярной кластерной вычислительной системой на основе Apache Hadoop, которая предлагает большую потенциальную ценность благодаря своей скорости и простоте использования. Мы рассмотрим его здесь, уделив особое внимание интерфейсу Python для Spark: PySpark.

Подготовка
-------------

Для работы нам потребуется собствено сам Spark который можно скачать и установить с официального сайта http://spark.apache.org/downloads.html

Так же для его успешного функционирования потребуется Java8/11. И вот тут могут возникнуть сложности, поскольку сейчас последняя и поддерживаемая верся - Java11, но самостоятельная настройка может вызывать затруднение при совместимости пакетов, такие как ошибка вида:

`Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe. : java.lang.IllegalArgumentException: Unsupported class file major version 55` 

в таком случае самый простой вариант запустить данный блокнот используя **Google Colab**. 



*Замечание 1:
При локальном запуске и запуске в virtualenv мы должны указать Spark использовать текущую версию Python, иначе она будет использовать системную версию Python по умолчанию. Вставьте это в свой код: `os.environ['PYSPARK_PYTHON'] = sys.executable`.*

*Замечание 2:
Spark имеет веб-интерфейс, который показывает запущенные задачи, выполняющиеся процессы и различную статистику. Запуская локально, это может наблюдать в интерфесе http://localhost:4040/.*

*Замечание 3:
Запуская же ноутбук в **colab** чтобы получить такую ссылку раскомментируйте ячейку ниже и запустите ее. По этой ссылке будет доступен аналог локального хоста только для облачного блокнота. По этой ссылке доступ будет только у вас, залогиненных в учетной записи google. Для других же пользователей эта ссылка будет выдавать 403 ошибку - ошибку доступа к ресурсу.*

In [ ]:
# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(4040)"))

## Поставим сам модуль

In [ ]:
!pip install pyspark

Обращения к pyspark
---------------

Чтобы вызвать Spark из Python, нам нужно использовать интерфейс PySpark. Например, его можно вызвать интерактивной оболочкой из вашей домашней директории Spark.:

    ./bin/pyspark

Как оболочка iPython Spark:

    IPYTHON=1 ./bin/pyspark

Или как пусковая установка для скриптов:

    ./bin/pyspark --master local

Ниже мы рассмотрим, как использовать API PySpark внутри скриптов Python.

In [1]:
import os
import sys

# Spark's home directory (here it's: ~/spark-1.6.0) should be set as an environment variable.
# (Of course setting an env. variable doesn't need to be done from Python; any method will do.)
# os.environ['SPARK_HOME'] = os.path.join(os.path.expanduser('~'), 'spark-1.6.0')

# Add Spark's Python interface (PySpark) to PYTHONPATH.
# (Again: this doesn't need to be done from Python.)
# sys.path.append(os.path.join(os.environ.get('SPARK_HOME'), 'python'))

# This can be useful for running in virtualenvs:
# os.environ['PYSPARK_PYTHON'] = '/home/nico/virtualenv/bin/python'

# OK, now we can import PySpark
from pyspark import SparkContext, SparkConf

Внутри нашей *рабочей программки* соединение с Spark представлено экземпляром `SparkContext`. Для локального запуска Spark вы можете просто создать его с помощью:

    sc = SparkContext('local', 'mySparkApp')

Кроме того, вы можете использовать экземпляр `SparkConf` для управления различными свойствами конфигурации Spark, что мы и будем рассматривать ниже.

In [7]:
conf = SparkConf()
conf.toDebugString()

'spark.app.name=spark_tutorial\nspark.cores.max=4\nspark.executor.memory=1g\nspark.master=local\nspark.submit.deployMode=client\nspark.ui.showConsoleProgress=true'

In [ ]:
# укажем что будем мы запускать все это локально
conf.setMaster('local')
conf.setAppName('spark_tutorial') # некоторый alias нашего "приложения"
# SparkConf имеет методы 'set', 'setAll' и 'setIfMissing' которые могут быть использованы
# для уточнения конфигурации нашего "кластера" - той части которую мы хотим заиспользовать
# например - задействовать 4 ядра и 1Gb оперативы
conf.setIfMissing("spark.cores.max", "4")
conf.set("spark.executor.memory", "1g")

In [ ]:
# Другой вариан, задать все это разом:
conf.setAll([('spark.cores.max', '4'), (("spark.executor.memory", "1g"))])

In [2]:
# И теперь запустим spark с такой конфигурацией
sc = SparkContext(conf=conf)

# остановить же это можно с помощью следующей команды в ручном режиме:
# sc.stop()

Как работает Spark, очень-очень вкратце
-------------------------

Spark использует *диспетчер кластеров* (например, собственный автономный менеджер Spark, YARN или Mesos) и несколько *рабочих узлов*. Менеджер задач (ака master/main) пытается получить *исполнителей* (ака slaves/secondary) на рабочих узлах, которые выполняют вычисления и хранят данные на основе кода и задач, которые им отправляются.


Основная абстракция Spark - это так называемый *Resilient Distributed Dataset (RDD)*. Spark может создавать RDD из любого источника хранения, поддерживаемого Hadoop. RDD содержит промежуточные результаты вычислений и хранится в ОЗУ или на диске на рабочих узлах. В случае сбоя узла, RDD может быть восстановлен. Многие процессы могут выполняться параллельно благодаря распределенной природе RDD, а конвейерная обработка и отложенное выполнение предотвращают необходимость сохранения промежуточных результатов для следующего шага. Важно отметить, что Spark поддерживает извлечение наборов данных в кластерный *кэш в памяти* для быстрого доступа.

Операции RDD можно разделить на 2 группы: *преобразования* (transform) и *действия* (actions). Преобразования (например, `map`) RDD всегда приводят к новым RDD, а действия (например, `reduce`) возвращают значения, которые являются результатом операций над RDD, обратно в программу драйвера.

Звучит сложно, но из кода попробуем понять это более наглядно

### RDD и распределенные данные

Сейчас, когда мы запускаем все это дело локально на одной машине - они в реалиях не очень то распределенные, они лежат на физическом одном диске. Однако даже в этом случае запускаясь локально Spark будет оркестрировать всем, как будто у него маленький кластер. Настолько маленький, что ровно из одного вашего компьютера =)

In [3]:
# 'parallelize' создает RDD путем распределения данных по кластеру
rdd = sc.parallelize(range(14), numSlices=4)
# по сути создаем список из 14 элементов которой храним распределенно, на 4 "файлах"
print("Number of partitions: {}".format(rdd.getNumPartitions()))
# 'glom' перечисляет все элементы в каждом разделе
print(rdd.glom().collect())

Number of partitions: 4


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalArgumentException: Unsupported class file major version 55
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:166)
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:148)
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:136)
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:237)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:50)
	at org.apache.spark.util.FieldAccessFinder$$anon$4$$anonfun$visitMethodInsn$7.apply(ClosureCleaner.scala:845)
	at org.apache.spark.util.FieldAccessFinder$$anon$4$$anonfun$visitMethodInsn$7.apply(ClosureCleaner.scala:828)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:134)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$for

### Spark - ленивый
Несмотря на любые промежуточные преобразования, Spark запускается только после выполнения *действия* на RDD. Это связано с тем, что он пытается выполнить умную конвейеризацию операций, чтобы не сохранять промежуточные результаты.

Этакий знакомый аналог `map` в питоне, который по факту еще ничего не применяет

In [6]:
rddSquared = rdd.map(lambda x: x ** 2)

print(rddSquared.collect())

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169]


In [ ]:
# Альтернативный вариант, с созданием функции:
def squared(x):
    return x ** 2
rddSquared = rdd.map(squared)
print(rddSquared.collect())


В данных обоих случаях только `collect` инициировал работу с данными, остальные же созданные преобразования откладывались как "состояния"

Рассмотрим другие популярные преобразования

In [ ]:
# Преобразования
# -----------------------

func = lambda x: -x
rdd.map(func)
rdd.flatMap(func) # почти как map, только результат будет распакован
rdd.filter(func)
rdd.sortBy(func)

In [ ]:
# действия
# ---------------

rdd.reduce(lambda x, y: x + y)
rdd.count()

В обоих этих случаях операции по сути никуда не применились, можно сказать что мы выстроили процесс по которому будут выполняться узлы, однако каждый из них вел в "никуда"

In [7]:
# Действия, возвращающие полученные в данные
print(rdd.collect())                    # вернуть все эллементы
print(rdd.first())                      # вернуть первый элемент
print(rdd.take(5))                      # вернуть первые N элементов
print(rdd.top(3))                       # Вернуть первые N элементов упорядоченные по убыванию
print(rdd.takeOrdered(7, lambda x: -x)) # Вернуть N эллементов, отсортированных согласно какой то "функции"

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
0
[0, 1, 2, 3, 4]
[13, 12, 11]
[13, 12, 11, 10, 9, 8, 7]


# Упражнение: Решето Эратосфена.

Напишите алгоритм просеивания простых чисел оперирую pyspark

Подсказка: все не так то просто, последовательные фильтры надо явно заставлять выполнять

In [8]:
n = 1000

# Your code here

### RDD может использовать кеши
Spark позволяет пользователю контролировать, какие данные и как кэшируются. Правильное кэширование RDD может быть чрезвычайно полезным! Всякий раз, когда у вас есть RDD, который будет использоваться повторно несколько раз, вам следует рассмотреть возможность его кэширования.

In [ ]:
import numpy as np

NUM_SAMPLES = int(1e6)
rddBig = sc.parallelize(np.random.random(NUM_SAMPLES))

# нет кэширования: будет пересчитываться каждый раз, когда мы проходим цикл
rddBigTrans = rddBig.map(lambda x: (x ** 2 - 0.1) ** 0.5)
print(rddBigTrans.getStorageLevel())
for threshold in (0.2, 0.4, 0.6, 0.8):
    %timeit -n 1 -r 1 rddBigTrans.filter(lambda x: x >= threshold).count()

In [ ]:
# мы кешируем этот промежуточный результат, потому что он будет неоднократно вызываться
rddBigTrans_c = rddBig.map(lambda x: (x ** 2 - 0.1) ** 0.5).cache()
print(rddBigTrans_c.getStorageLevel())
for threshold in (0.2, 0.4, 0.6, 0.8):
    %timeit -n 1 -r 1 rddBigTrans_c.filter(lambda x: x >= threshold).count()

In [8]:
# используем unpersist для удаления из кэша
print(rddBigTrans_c.unpersist().getStorageLevel())
# для еще более детального управления кэшированием используйте функцию «persist» 
from pyspark import storagelevel
print(rddBigTrans.persist(storagelevel.StorageLevel.MEMORY_AND_DISK).getStorageLevel())

ImportError: No module named 'numpy'

### Spark: key-value хранилище
Так называемые PairRDD - это RDD, в которых хранятся пары ключ-значение. В Spark используется множество специальных операций, таких как объединение по ключу, группирование по ключу и т. д.

In [ ]:
# PairRDD автоматически создаются всякий раз, когда мы представляем список кортежей ключ-значение
# Здесь мы трансформируем rddA и создаем ключ на основе четных / нечетных флагов.
rddP1 = rdd.map(lambda x: (x % 2 == 0, x))

In [ ]:
# Более понятный вариант для этого:
rddP1 = rdd.keyBy(lambda x: x % 2 == 0)

In [ ]:
# Другой способ создать PairRDD - это заархивировать два RDD (предполагается, что RDD одинаковой длины)
print("Zipped: {}".format(rdd.zip(rdd).collect()))

In [ ]:
# Доступ к ключам и значениям
print("Keys: {}".format(rddP1.keys().collect()))
print("Values: {}".format(rddP1.values().collect()))

In [ ]:
# Другой вариант обращения к ключам-значением - через кортеж; x[0] - key, x[1] - value
print(rddP1.map(lambda x: (x[0], x[1] ** 2)).collect())

In [ ]:
# Лучше: mapValues / flatMapValues, который работает только со значениями и сохраняет ключи на месте
print(rddP1.mapValues(lambda x: x ** 2).collect())

In [ ]:
# Мы также можем вернуться от PairRDD к обычному RDD, просто опустив ключ
print(rddP1.map(lambda x: x[1] ** 2).collect())

In [ ]:
# Возможны различные агрегации по ключу, такие как reduceByKey, combineByKey и foldByKey
# пример с reduceByKey:
print("Sum per key: {}".format(rddP1.reduceByKey(lambda x, y: x + y).collect()))

In [ ]:
# Кроме того, некоторые общие операции доступны в форме «ByKey», например:
rddP1.sortByKey()
rddP1.countByKey()

# Группировка и соединение по ключу

In [ ]:
# Существуют различные возможные способы объединения двух RDD по ключу:
rddP2 = sc.parallelize(range(0, 28, 2)).map(lambda x: (x % 2 == 0, x))

In [ ]:
# inner join / cross join в случае наложения ключей
print("Join: {}".format(rddP1.join(rddP2).collect()))

In [ ]:
# left/right outer join
rddP1.leftOuterJoin(rddP2)
rddP1.rightOuterJoin(rddP2)

In [ ]:
# для всех ключей в rddP1 или rddP2 cogroup возвращает итерируемые значения
print("Cogroup: {}".format(rddP1.cogroup(rddP2).collect()))
# Группируем вместе более двух RDD по ключу можно с помощью groupWith
rddP1.groupWith(rddP2, rddP2)

# с groupByKey мы создаем новый RDD, который сохраняет те же ключи на том же узле, где это возможно
print("After groupByKey: {}".format(rddP1.groupByKey().glom().collect()))

### Spark: работа напряму с созданием фреймов RDD из текстовых файлов

In [ ]:
# TODO: Вариант для colab, локально можно поискать другие удобные файлы
from pyspark import SparkFiles
sc.addFile(os.path.join('/content/sample_data', 'README.md'))
rddT = sc.textFile(SparkFiles.get('README.md'))
print(rddT.take(5))

### RDDs простые статистические аггрегаторы

In [ ]:
print(rdd.stats())
print(rdd.count())
print(rdd.sum())
print(rdd.mean())
print(rdd.stdev(), rdd.sampleStdev())
print(rdd.variance(), rdd.sampleVariance())
print(rdd.min(), rdd.max())
print(rdd.histogram(5))

### RDDs преобразования множеств

In [ ]:
rddB = sc.parallelize(range(0, 26, 2))
print(rdd.union(rddB).collect()) # or: rdd + rddB
print(rdd.union(rddB).distinct().collect())
print(rdd.intersection(rddB).collect())
print(rdd.subtract(rddB).collect())
print(rdd.cartesian(rddB).collect())

### Spark поддержка общих переменных

In [ ]:
# Общая переменная копируется на каждую машину только один раз, эффективным образом.
# Это очень удобно, когда каждый узел использует данные в нем, и особенно, если данные
# большие и в противном случае будут отправлены по сети несколько раз.
broadcastVar = sc.broadcast({'CA': 'California', 'NL': 'Netherlands'})
print(broadcastVar.value)

In [ ]:
# "Аккумулятор" является общей переменной, которая живет на главном узле,
# который каждая операция может просматривать.
accu = sc.accumulator(0)

In [ ]:
# 'foreach' просто применяет функцию к каждому элементу RDD, ничего не возвращая
rdd.foreach(lambda x: accu.add(x))
print(accu.value)

Популярные баги:
--------

### Не кэшировать промежуточные результаты, которые  используются позже

In [ ]:
print("Not so great:")
rddBigTrans = rddBig.map(lambda x: (x ** 2 - 0.1) ** 0.5)
for threshold in (0.2, 0.4, 0.6, 0.8):
    %timeit -n 1 -r 1 rddBigTrans.filter(lambda x: x >= threshold).count()

In [ ]:
print("Better:")
rddBigTrans_c = rddBig.map(lambda x: (x ** 2 - 0.1) ** 0.5).cache()
for threshold in (0.2, 0.4, 0.6, 0.8):
    %timeit -n 1 -r 1 rddBigTrans_c.filter(lambda x: x >= threshold).count()

### Не учитывать, когда и как данные передаются через кластер
Имейте в виду, что Spark является распределенной вычислительной средой и что следует избегать передачи данных по сети внутри кластера (пропускная способность сети в ~100 раз дороже пропускной способности памяти).

In [ ]:
# groupByKey запускает случайное воспроизведение, поэтому по сети копируется много данных
sumPerKey = rddP1.groupByKey().mapValues(lambda x: sum(x)).collect()

In [ ]:
# Лучше: reduceByKey уменьшает ту передачу локально перед "перетасовкой"
sumPerKey = rddP1.reduceByKey(lambda x, y: x + y).collect()

### Не работать с соответствующим количеством разделов

Недостаточное количество разделов приводит к плохому параллелизму в кластере.

Это также оказывает нагрузку на память для определенных операций.

In [ ]:
# С другой стороны, предположим, что RDD распределен по 1000 разделам,
# но мы работаем только над небольшим подмножеством данных в RDD, например:
rddF = rdd.filter(lambda x: x < 0.1).map(lambda x: x ** 2)

In [ ]:
# Затем мы эффективно создаем много пустых задач и используем объединение или перераспределение.
# было бы полезно создать RDD с меньшим количеством разделов
rddF = rdd.filter(lambda x: x < 3).coalesce(10).map(lambda x: x ** 2)

### Используя преобразование с высокими накладными расходами на элемент, лучше использовать mapPartitions

In [ ]:
# Например, просто подключение к базе и отключени от нее уже требует расходов
def db_operation(x):
    # тут мы подключилис
    # Поделали что-то с элементом
    # завершаем действие, отключаемся от базы
    pass

# Особенно, если вы повторите это для каждого элемента:
rdd.map(db_operation)

In [ ]:
# Лучше: делайте это на уровне раздела, а не на уровне элемента.
def vectorized_db_operation(x):
    # тут мы подключились
    # Поделали что-то с элементом
    # завершаем действие, отключаемся от базы
    pass

# в таком случае мы будем обрабатывать даныне целиком пачками, они все будут вычитывать в память за раз
result = rdd.mapPartitions(vectorized_db_operation)

### Отправка большого количества данных вместе с вызовом функции для каждого элемента

In [ ]:
bigData = np.random.random(int(1e6)) #наши "большие данные", мы ж все же на одной машинке работаем

def myFunc(x):
    return x * np.random.choice(bigData)

# и тогда наш массив будет отправляться в каждую партицию, то есть просто гоняться по сети в холостую
rdd.map(myFunc)

# Лучше: сделать большие данные доступными только для чтения, чтобы они эффективно копировались по сети
bigDataBC = sc.broadcast(bigData)

В боевых задачах
-------------

### Поучим что-нибудь скайлерном

Рассмотрим полусинтетический пример. Создадим какую нибудь выборку данных, из которых мы захотим решить задачу регересси.

Вот только решение ее - будем делать распределенно.

In [ ]:
from sklearn.cross_validation import train_test_split, ShuffleSplit
from sklearn.datasets import make_regression
from sklearn import pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

N = 10000   # number of data points
D = 100     # number of dimensions

X, y = make_regression(
    n_samples=N,
    n_features=D,
    n_informative=int(D*0.1),
    n_targets=1,
    bias=-6.,
    noise=50.,
    random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# раскидаем данные случаным образом по партициям
samples = sc.parallelize(ShuffleSplit(y_train.size, n_iter=8))
reg_model = pipeline.Pipeline([("scaler", StandardScaler()), ("ridge", Ridge())])
# это кусочек обработки данных для обучения - перегоним переменные в нормальное распределение нормировкой,
# и потом будем запусать на них решение задачи гребневой регрессии

# обучим модель на каждой пачке и примеyим к выборке
mean_rsq = samples.map(
    lambda (index, _): reg_model.fit(X[index], y[index]).score(X_test, y_test)
).mean()
print(mean_rsq)

получили такой самопальный вариант нескольких моделей, которые как-то голосуют за данные

# Упражнение: "не боевая" работа "с боевыми" данными

Нам потребуется датасет с ценами на жилье https://www.kaggle.com/camnugent/california-housing-prices
попробуем пообрабатывать его не привычным пандасом, а с использованием спарка

# Задание 1

считаем датасет и приведем его в человеческий вид

In [10]:
from pyspark.sql import Row
# поможет нам собрать из строчек более привычный пандасовский вариант
# здесь нам лучше избавиться пока от заголовка в файле,
# зато сделать данные более удобными назначива постолбцовое хранение
rdd = sc.textFile('/content/sample_data/california_housing_train.csv')
pass



# Задание 2 

теперь проведем все колонки в типизированный вид, там же пока строки

In [11]:
from pyspark.sql.types import *


# Задание 3

Добавим новых признаков:
    * комнат на домовладельцев
    * жителей на домовладение
    * доля спальных комнат относительно всех

In [ ]:
from pyspark.sql.functions import *


# Стахостический градиентный спуск своими руками, как бонус

### Stochastic gradient descent using scikit-learn (from: https://gist.github.com/MLnick/4707012)
Each partition is a mini-batch for the SGD, uses average weights.

In [ ]:
from sklearn import linear_model as lm
from sklearn.base import copy

N = 10000   # Number of data points
D = 10      # Numer of dimensions
ITERATIONS = 5
np.random.seed(seed=42)

def generate_data(N):
    return [[[1] if np.random.rand() < 0.5 else [0], np.random.randn(D)]
            for _ in range(N)]

def train(iterator, sgd):
    for x in iterator:
        sgd.partial_fit(x[1], x[0], classes=np.array([0, 1]))
    yield sgd

def merge(left, right):
    new = copy.deepcopy(left)
    new.coef_ += right.coef_
    new.intercept_ += right.intercept_
    return new

def avg_model(sgd, slices):
    sgd.coef_ /= slices
    sgd.intercept_ /= slices
    return sgd

slices = 4
data = generate_data(N)
print(len(data))

# init stochastic gradient descent
sgd = lm.SGDClassifier(loss='log')
# training
for ii in range(ITERATIONS):
    sgd = sc.parallelize(data, numSlices=slices) \
            .mapPartitions(lambda x: train(x, sgd)) \
            .reduce(lambda x, y: merge(x, y))
    # averaging weight vector => iterative parameter mixtures
    sgd = avg_model(sgd, slices)
    print("Iteration %d:" % (ii + 1))
    print("Model: ")
    print(sgd.coef_)
    print(sgd.intercept_)

The Spark universe
------------------

Other interesting tools for Spark:

- Spark SQL: http://spark.apache.org/docs/latest/sql-programming-guide.html
- MLlib, Spark's machine learning library: http://spark.apache.org/docs/latest/mllib-guide.html
- Spark Streaming, for streaming data applications: http://spark.apache.org/docs/latest/streaming-programming-guide.html

More information
----------------

### Documentation

Spark documentation: https://spark.apache.org/docs/latest/index.html

Spark programming guide: http://spark.apache.org/docs/latest/programming-guide.html

PySpark documentation: https://spark.apache.org/docs/latest/api/python/index.html

### Books

Learning Spark: http://shop.oreilly.com/product/0636920028512.do

(preview: https://www.safaribooksonline.com/library/view/learning-spark/9781449359034/)

### Talks (recommended to watch them in this order)

Parallel programming with Spark: https://www.youtube.com/watch?v=7k4yDKBYOcw

Advanced Spark features: https://www.youtube.com/watch?v=w0Tisli7zn4

PySpark: Python API for Spark: https://www.youtube.com/watch?v=xc7Lc8RA8wE

Understanding Spark performance: https://www.youtube.com/watch?v=NXp3oJHNM7E

A deeper understanding of Spark's internals: https://www.youtube.com/watch?v=dmL0N3qfSc8